### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Tourette_Syndrome/GSE146509'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

import pandas as pd

# Ensure clinical_data is correctly defined beforehand
clinical_data = {
    0: ['subject status: Arnold-Chiari (AC)', 'subject status: ACTS', 'subject status: unaffected controls (NC)', 'subject status: Tourette syndromes (TS)'],
    1: ['age: 14', 'age: 16', 'age: 9', 'age: 8', 'age: 11', 'age: 12', 'age: 10', 'age: 17', 'age: 15', 'age: 5', 'age: 13'],
    2: ['gender: M', 'gender: F'],
    3: ['tissue: serum'],
    4: ['molecule subtype: circulating RNA']
}

# Convert dictionary to DataFrame
clinical_data_df = pd.DataFrame.from_dict(clinical_data, orient='index').transpose()

# 1. Confirming Gene Expression Data
is_gene_available = False

# 2. Variable Availability and Data Type Conversion
trait_row = 0 if len({val.split(': ')[1] for val in clinical_data[0]}) > 1 else None
age_row = 1 if len({val.split(': ')[1] for val in clinical_data[1]}) > 1 else None
gender_row = 2 if len({val.split(': ')[1] for val in clinical_data[2]}) > 1 else None

# Conversion Functions
def convert_trait(value):
    try:
        val = value.split(': ')[1]
        return 1 if val == 'Tourette syndromes (TS)' else 0
    except (IndexError, ValueError):
        return None

def convert_age(value):
    try:
        return float(value.split(': ')[1])
    except (IndexError, ValueError):
        return None

def convert_gender(value):
    try:
        val = value.split(': ')[1]
        return 1 if val == 'M' else 0 if val == 'F' else None
    except (IndexError, ValueError):
        return None

# 3. Save Metadata
save_cohort_info('GSE146509', './preprocessed/Tourette_Syndrome/cohort_info.json', is_gene_available, trait_row is not None)

# 4. Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data_df, 'Tourette_Syndrome', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Tourette_Syndrome/trait_data/GSE146509.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
